This jupyter notebook shows the training process of labeled dataset of video using transformer encoder.
<b>
At the end of the we also tested the trained model with test dataset to evaluate the performance.

In [2]:
from model import Encoder,pos_encoding
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Dense,GlobalMaxPool1D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from tensorflow.keras import Input
from tensorflow.keras import Model
import cv2 as cv


In [3]:
seed_constant = 50
np.random.seed(seed_constant)

tf.random.set_seed(seed_constant)

### loading training labels and labels

In [4]:
features=np.load('../features/features.npy')
labels=np.load('../features/labels.npy')

### Splitting the dataset in train and test in ration 0.8 and 0.2 respectively

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

### Settng the  hyperparameters for training the model

In [6]:
num_class=4
d_model=features.shape[2]
lr=0.001
dff=4
num_heads=1
num_layers=1
seq_len=20
num_epochs=200


In [7]:
## parameter for testing
IMG_SIZ=128

In [8]:
## categorical representation of y_train
y_train=to_categorical(y_train,num_class)
y_test=to_categorical(y_test,num_class)

### Initializing the encodder object

In [9]:
encodder=pos_encoding()

In [10]:
def model():
    inputs=Input(shape=(seq_len,d_model))
    x=pos_encoding()(inputs)
    x=Encoder(num_layers=num_layers,num_heads=num_heads,d_model=d_model,dff=dff,rate=0.3)(x,mask=None,training=True)
    x=GlobalMaxPool1D()(x)
    x=Dense(units=num_class,activation='softmax')(x)
    model=Model(inputs,x,name='Transfromer_Encoder')
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy',metrics=['accuracy'])
    return model


In [11]:
training_model=model()

2022-11-19 11:47:39.944417: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 11:47:39.962669: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 11:47:39.962825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 11:47:39.963359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [12]:
training_model.summary()

Model: "Transfromer_Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 1024)]        0         
                                                                 
 pos_encoding_1 (pos_encodin  (None, 20, 1024)         20480     
 g)                                                              
                                                                 
 encoder (Encoder)           (None, 20, 1024)          4211716   
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dense_6 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 4,236,296
Trainable params: 4,236

### Setting the callbacks: EarlyStopping and tensorboard callbacks

In [13]:
log_dire='../logs'
#earlystopping=EarlyStopping(monitor='val_accuracy',mode='min',restore_best_weights=True,patience=20)
tb_callback=TensorBoard(log_dir=log_dire) ## tensorboard callback

In [14]:
history=training_model.fit(x=X_train,y=y_train,validation_split=0.2,verbose=1,epochs=num_epochs,shuffle=True,
             callbacks=[tb_callback])

Epoch 1/200
8/8 [==============================] - 1s 38ms/step - loss: 4.1221 - accuracy: 0.2773 - val_loss: 1.6515 - val_accuracy: 0.5156
Epoch 2/200
8/8 [==============================] - 0s 15ms/step - loss: 1.1794 - accuracy: 0.5273 - val_loss: 0.7957 - val_accuracy: 0.7188
Epoch 3/200
8/8 [==============================] - 0s 15ms/step - loss: 0.8558 - accuracy: 0.6680 - val_loss: 0.6589 - val_accuracy: 0.8438
Epoch 4/200
8/8 [==============================] - 0s 14ms/step - loss: 0.6130 - accuracy: 0.7461 - val_loss: 0.6114 - val_accuracy: 0.8125
Epoch 5/200
8/8 [==============================] - 0s 14ms/step - loss: 0.4405 - accuracy: 0.8711 - val_loss: 0.3969 - val_accuracy: 0.8750
Epoch 6/200
8/8 [==============================] - 0s 15ms/step - loss: 0.2766 - accuracy: 0.9219 - val_loss: 0.2900 - val_accuracy: 0.9219
Epoch 7/200
8/8 [==============================] - 0s 14ms/step - loss: 0.1591 - accuracy: 0.9531 - val_loss: 0.2249 - val_accuracy: 0.9219
Epoch 8/200
8/8 [===

In [15]:
loss,test_accuracy=training_model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 7ms/step - loss: 0.3023 - accuracy: 0.9430


Saving the Model weight

In [ ]:
training_model.save_weights('../saved_weight/', overwrite=True, save_format=None, options=None)